# EAS: Predicción

En este notebook mostramos como aplicar un modelo ya entrenado para predecir sobre un área en particular. En este caso vamos a mostrar cómo predecir sobre las imágenes de una remesa.

## Preparación de datos

![](img/remesas.jpg)

En este notebook vamos a mostrar como predecir sobre una remesa en particular (la 5).

In [1]:
import os

In [6]:
DATA_DIR = "data/paneles"
REMESA_DIR = os.path.join(DATA_DIR, 'predict', 'remesa5')

El primer paso sería el de la descarga de las imágenes para la remesa que queremos analizar. Podemos usar el shapefile de Remesas que se puede descargar en IDE, filtrar por la remesa que buscamos, y usar el paquete `ideuy` para filtrar la grilla y descargar las imágenes. Este paso lo omitimos por brevedad, pero ante la duda consultar el notebook 1 y 2.

En general, recomendamos trabajar con áreas como las Remesas o cualquier otro polígono de tamaño similar (o más pequeño), antes que analizar *todo* el país de una pasada, porque puede llegar a ser un proceso muy largo y es más dificil de seguir paso a paso en caso de que haya una falla o un problema en el camino.

## Preprocesamiento

Una vez que tenemos las imágenes del área de análisis, debemos generar los chips sobre los cuales se hará la predicción.

A diferencia de la extracción de chips en el entrenamiento, no generaremos máscaras, porque justamente no tenemos esa información (es lo que queremos predecir!). Además, usamos como `--step-size` el mismo valor que `--size`, porque no nos interesa generar chips con solapamiento. Tampoco usamos la opción `--aoi`, porque queremos analizar toda la imagen entera y para entrenar nos interesaban solamente los chips que tuvieran algún objeto.

Como ejemplo, suponiendo que las imágenes fueron descargadas en `data/raw_images/urban/` y `data/raw_images/national` respectivamente, y que la remesa que queremos analizar es la 5, los comandos para las imágenes urbanas y las nacionales serían las siguientes:

In [6]:
# modelo 1: urbano
!meduy_extract_chips \
    data/raw_images/urban/*Remesa_05*.jpg \
    -o $REMESA_DIR/urban/400_400/ \
    --size 400 \
    --step-size 400 \
    --crs epsg:5382

 36%|███████████████▋                            | 5/14 [00:20<00:35,  3.98s/it]/home/ro/miniconda3/envs/tf-gpu/lib/python3.7/site-packages/rasterio/__init__.py:218: NotGeoreferencedWarning: Dataset has no geotransform set. The identity matrix may be returned.
  s = DatasetReader(path, driver=driver, sharing=sharing, **kwargs)

  0%|                                                   | 0/575 [00:00<?, ?it/s]/home/ro/miniconda3/envs/tf-gpu/lib/python3.7/site-packages/rasterio/__init__.py:232: NotGeoreferencedWarning: The given matrix is equal to Affine.identity or its flipped counterpart. GDAL may ignore this matrix and save no geotransform without raising an error. This behavior is somewhat driver-specific.
  **kwargs)

 35%|██████████████▎                          | 201/575 [00:01<00:04, 76.89it/s]


 51%|████████████████████▉                    | 294/575 [00:03<00:03, 77.69it/s]


100%|███████████████████████████████████████████| 14/14 [01:11<00:00,  5.11s/it]


In [7]:
# modelo 2: nacional
!meduy_extract_chips \
    data/raw_images/national/*Remesa_05*.jpg \
    -o $REMESA_DIR/national/160_160/ \
    --size 160 \
    --step-size 160 \
    --crs epsg:5382

 58%|█████████████████████▎               | 7089/12285 [00:27<00:20, 254.50it/s]


 15%|█████▌                               | 1861/12285 [00:07<00:36, 288.76it/s]


 75%|███████████████████████████▊         | 9247/12285 [00:35<00:13, 221.81it/s]


 36%|█████████████▏                       | 4370/12285 [00:16<00:29, 265.95it/s]


 96%|██████████████████████████████████▋ | 11837/12285 [00:44<00:02, 220.64it/s]


...

 88%|███████████████████████████████▊    | 10868/12285 [00:42<00:05, 276.10it/s]


 47%|█████████████████▎                   | 5741/12285 [00:21<00:29, 221.89it/s]


  4%|█▎                                    | 434/12285 [00:01<01:06, 176.94it/s]


 62%|██████████████████████▊              | 7593/12285 [00:29<00:16, 285.44it/s]


100%|███████████████████████████████████████████| 15/15 [11:53<00:00, 47.60s/it]


```
data/paneles/predict
└── remesa5
    ├── national
    │   └── 160_160
    │       └── images
    └── urban
        └── 400_400
            └── images
```

## Predicción

In [3]:
import warnings
warnings.filterwarnings('ignore')

from meduy.unet.predict import PredictConfig, predict

Using TensorFlow backend.


In [11]:
DATA_DIR

'data/paneles'

In [12]:
predict_config = PredictConfig(
    images_path=os.path.join(REMESA_DIR, 'urban', '400_400'),
    results_path=os.path.join(REMESA_DIR, 'urban', '400_400', 'results'),
    batch_size=32,
    model_path=os.path.join(DATA_DIR, 'weights', 'paneles_m1_v1.h5'),
    height=320,
    width=320,
    n_channels=3,
    n_classes=1)

predict(predict_config)  # Ejecuta la predicción

  0%|          | 0/105 [00:00<?, ?it/s]

100%|██████████| 105/105 [00:50<00:00,  2.06it/s]

Done!


Los parámetros de configuración para la predicción son similares a los de entrenamiento, salvo por algunos:

* `images_path`: ruta a las imagenes sobre las cuales queremos predecir
* `results_path`: ruta donde se almacenarán los chips resultado
* `model_path`: ruta al archivo de pesos (.h5) del modelo entrenado
* `batch_size`: tamaño de cada batch (idem entrenamiento)
* `width`, `height`: ancho y alto de los chips en la red (idem entrenamiento)
* `n_channels`: número de canales, RGB = 3 (idem entrenamiento)
* `n_classes`: cantidad de clases, en este caso 1 (idem entrenamiento)

In [14]:
predict_config_national = PredictConfig(
    images_path=os.path.join(REMESA_DIR, 'national', '160_160'),
    results_path=os.path.join(REMESA_DIR, 'national', '160_160', 'results'),
    batch_size=64,
    model_path=os.path.join(DATA_DIR, 'weights', 'paneles_m2b_v3.h5'),
    height=160,
    width=160,
    n_channels=3,
    n_classes=1)

predict(predict_config_national)  # Ejecuta la predicción

100%|██████████| 2885/2885 [20:13<00:00,  2.38it/s]

Done!
